In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Install required packages\n",
    "!pip install --upgrade transformers datasets torch peft sklearn tqdm huggingface-hub\n",
    "\n",
    "# Log in to Hugging Face (run this cell and paste your token when prompted)\n",
    "!huggingface-cli login\n",
    "\n",
    "# Import libraries\n",
    "import os\n",
    "import torch\n",
    "from transformers import AutoTokenizer, AutoModelForCausalLM\n",
    "from datasets import load_dataset\n",
    "from sklearn.metrics import accuracy_score\n",
    "import tqdm\n",
    "from peft import PeftModel\n",
    "\n",
    "# Define cache directory for pre-trained models\n",
    "cache_dir = \"./pretrained-models\"\n",
    "os.makedirs(cache_dir, exist_ok=True)\n",
    "\n",
    "# Load the FinGPT-Forecaster dataset (Dow 30 dataset)\n",
    "dataset = load_dataset(\"FinGPT/fingpt-forecaster-dow30-202305-202405\", split=\"train\")\n",
    "print(f\"Dataset loaded with {len(dataset)} rows\")\n",
    "print(dataset.column_names)  # Should show 'prompt', 'answer', 'period', 'label', 'symbol'\n",
    "\n",
    "# Load Llama3-1-8B base model (authenticated via huggingface-cli login)\n",
    "llama3_base = AutoModelForCausalLM.from_pretrained(\n",
    "    \"meta-llama/Llama-3.1-8B\",\n",
    "    trust_remote_code=True,\n",
    "    device_map=\"auto\",\n",
    "    cache_dir=cache_dir,\n",
    "    torch_dtype=torch.float16,\n",
    ")\n",
    "\n",
    "# Load DeepSeek base model (assuming it’s accessible)\n",
    "deepseek_base = AutoModelForCausalLM.from_pretrained(\n",
    "    \"deepseek-ai/DeepSeek-R1-Distill-Llama-8B\",\n",
    "    trust_remote_code=True,\n",
    "    device_map=\"auto\",\n",
    "    cache_dir=cache_dir,\n",
    "    torch_dtype=torch.float16,\n",
    ")\n",
    "\n",
    "# Load tokenizers for both models\n",
    "llama3_tokenizer = AutoTokenizer.from_pretrained(\"meta-llama/Llama-3.1-8B\", cache_dir=cache_dir)\n",
    "deepseek_tokenizer = AutoTokenizer.from_pretrained(\"deepseek-ai/DeepSeek-R1-Distill-Llama-8B\", cache_dir=cache_dir)\n",
    "\n",
    "# Data Preprocessing and Filtering\n",
    "def filter_by_ticker(test_dataset, ticker_code):\n",
    "    filtered_data = []\n",
    "    for row in test_dataset:\n",
    "        prompt_content = row['prompt']\n",
    "        ticker_symbol = re.search(r\"ticker['s]?[(A-Z)+]\", prompt_content)\n",
    "        if ticker_symbol and ticker_symbol.group(1) == ticker_code:\n",
    "            filtered_data.append(row)\n",
    "    filtered_dataset = Dataset.from_dict({key: [row[key] for row in filtered_data] for key in test_dataset.column_names})\n",
    "    return filtered_dataset\n",
    "\n",
    "def get_unique_ticker_symbols(test_dataset):\n",
    "    ticker_symbols = set()\n",
    "    for i in range(len(test_dataset)):\n",
    "        prompt_content = test_dataset[i]['prompt']\n",
    "        ticker_symbol = re.search(r\"ticker['s]?[(A-Z)+]\", prompt_content)\n",
    "        if ticker_symbol:\n",
    "            ticker_symbols.add(ticker_symbol.group(1))\n",
    "    return list(ticker_symbols)\n",
    "\n",
    "def insert_guidance_after_intro(prompt):\n",
    "    intro_marker = \"You are a seasoned stock market analyst. Your task is to list the positive developments and \"\n",
    "    guidance = \"potential concerns for companies based on relevant news and financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week.\"\n",
    "    \n",
    "    intro_pos = prompt.find(intro_marker)\n",
    "    guidance_start_pos = prompt.find(guidance, intro_pos)\n",
    "    guidance_end_pos = guidance_start_pos + len(guidance) if guidance_start_pos != -1 else -1\n",
    "    \n",
    "    if intro_pos == -1 or guidance_start_pos == -1 or guidance_end_pos == -1:\n",
    "        return prompt\n",
    "    \n",
    "    guidance_section = prompt[guidance_start_pos:guidance_end_pos].strip()\n",
    "    new_prompt = (\n",
    "        prompt[:intro_pos + len(intro_marker)] + \"\\n\" +\n",
    "        guidance + \"\\n\" +\n",
    "        prompt[guidance_end_pos:]\n",
    "    )\n",
    "    return new_prompt\n",
    "\n",
    "import re\n",
    "from datasets import Dataset\n",
    "\n",
    "test_dataset = dataset.filter(lambda x: x[\"prompt\"], input_columns=[\"prompt\"])\n",
    "test_dataset = test_dataset.map(lambda x: {\"prompt\": insert_guidance_after_intro(x[\"prompt\"])})\n",
    "filtered_dataset = filter_by_ticker(test_dataset, \"AXP\")\n",
    "print(f\"Filtered dataset for AXP has {len(filtered_dataset)} rows\")\n",
    "\n",
    "# LoRA Fine-Tuning for Both Models\n",
    "from peft import LoraConfig, get_peft_model\n",
    "\n",
    "lora_config = LoraConfig(\n",
    "    r=8,  # Rank of the adaptation\n",
    "    lora_alpha=16,\n",
    "    lora_dropout=0.1,\n",
    "    target_modules=[\"q_proj\", \"v_proj\"],  # Adjust based on Llama’s architecture\n",
    "    task_type=\"CAUSAL_LM\",\n",
    ")\n",
    "\n",
    "# Fine-tune Llama-3.1-8B\n",
    "llama3_model = get_peft_model(llama3_base, lora_config)\n",
    "llama3_model.load_adapter(\n",
    "    \"./finetuned_models/dow30-202305-202405-llama-3.1-8b_202602280001\",  # Placeholder path\n",
    "    cache_dir=cache_dir,\n",
    ")\n",
    "llama3_model = llama3_model.eval()\n",
    "\n",
    "# Fine-tune DeepSeek-R1-Distill-Llama-8B\n",
    "deepseek_model = get_peft_model(deepseek_base, lora_config)\n",
    "deepseek_model.load_adapter(\n",
    "    \"./finetuned_models/dow30-202305-202405-DeepSeek-R1-Distill-llama-8B_202592020557\",\n",
    "    cache_dir=cache_dir,\n",
    ")\n",
    "deepseek_model = deepseek_model.eval()\n",
    "\n",
    "# Testing and Evaluation Function\n",
    "def test_demo(model, tokenizer, prompt):\n",
    "    inputs = tokenizer(prompt, return_tensors=\"pt\", padding=False, max_length=8000)\n",
    "    inputs = {key: value.to(model.device) for key, value in inputs.items()}\n",
    "    \n",
    "    start_time = time.time()\n",
    "    outputs = model.generate(\n",
    "        **inputs,\n",
    "        max_length=4096,\n",
    "        do_sample=True,\n",
    "        eos_token_id=tokenizer.eos_token_id,\n",
    "        use_cache=True,\n",
    "    )\n",
    "    end_time = time.time()\n",
    "    \n",
    "    output = tokenizer.decode(outputs[0], skip_special_tokens=True)\n",
    "    return output, end_time - start_time\n",
    "\n",
    "def test_acc(test_dataset, model_name):\n",
    "    answers_base, answers_fine_tuned, gts, times_base, times_fine_tuned = [], [], [], [], []\n",
    "    \n",
    "    if model_name == \"llama3\":\n",
    "        base_model = llama3_base\n",
    "        fine_tuned_model = llama3_model\n",
    "        tokenizer = llama3_tokenizer\n",
    "    elif model_name == \"deepseek\":\n",
    "        base_model = deepseek_base\n",
    "        fine_tuned_model = deepseek_model\n",
    "        tokenizer = deepseek_tokenizer\n",
    "    \n",
    "    for i in tqdm(range(len(test_dataset)), desc=\"Processing test samples\"):\n",
    "        try:\n",
    "            prompt = test_dataset[i][\"prompt\"]\n",
    "            gt = test_dataset[i][\"answer\"]\n",
    "            \n",
    "            output_base, time_base = test_demo(base_model, tokenizer, prompt)\n",
    "            answer_base = re.sub(r\"\\[INST\\]\\s*|\\s*\\[/INST\\]\", \"\", output_base, flags=re.DOTALL)\n",
    "            \n",
    "            output_fine_tuned, time_fine_tuned = test_demo(fine_tuned_model, tokenizer, prompt)\n",
    "            answer_fine_tuned = re.sub(r\"\\[INST\\]\\s*|\\s*\\[/INST\\]\", \"\", output_fine_tuned, flags=re.DOTALL)\n",
    "            \n",
    "            answers_base.append(answer_base)\n",
    "            answers_fine_tuned.append(answer_fine_tuned)\n",
    "            gts.append(gt)\n",
    "            times_base.append(time_base)\n",
    "            times_fine_tuned.append(time_fine_tuned)\n",
    "        except Exception as e:\n",
    "            print(f\"Error processing sample {i}: {e}\")\n",
    "            return answers_base, answers_fine_tuned, gts, times_base, times_fine_tuned\n",
    "    \n",
    "    return answers_base, answers_fine_tuned, gts, times_base, times_fine_tuned\n",
    "\n",
    "# Test both models\n",
    "llama3_answers_base, llama3_answers_fine_tuned, llama3_gts, llama3_base_times, llama3_fine_tuned_times = test_acc(filtered_dataset, \"llama3\")\n",
    "deepseek_answers_base, deepseek_answers_fine_tuned, deepseek_gts, deepseek_base_times, deepseek_fine_tuned_times = test_acc(filtered_dataset, \"deepseek\")\n",
    "\n",
    "# Metrics Calculation and Comparison\n",
    "def calc_metrics(predictions, ground_truths):\n",
    "    predictions_clean = [pred.strip().lower() for pred in predictions]\n",
    "    ground_truths_clean = [gt.strip().lower() for gt in ground_truths]\n",
    "    accuracy = accuracy_score(ground_truths_clean, predictions_clean)\n",
    "    return {\"accuracy\": accuracy}\n",
    "\n",
    "# Calculate metrics for Llama3\n",
    "llama3_base_metrics = calc_metrics(llama3_answers_base, llama3_gts)\n",
    "llama3_fine_tuned_metrics = calc_metrics(llama3_answers_fine_tuned, llama3_gts)\n",
    "\n",
    "# Calculate metrics for DeepSeek\n",
    "deepseek_base_metrics = calc_metrics(deepseek_answers_base, deepseek_gts)\n",
    "deepseek_fine_tuned_metrics = calc_metrics(deepseek_answers_fine_tuned, deepseek_gts)\n",
    "\n",
    "# Compare metrics\n",
    "comparing_metrics = calc_metrics(llama3_answers_fine_tuned, deepseek_answers_fine_tuned)\n",
    "\n",
    "# Print results\n",
    "print(\"Evaluating Base Model...\")\n",
    "print(\"Llama3 Base Metrics:\", llama3_base_metrics)\n",
    "print(\"DeepSeek Base Metrics:\", deepseek_base_metrics)\n",
    "print(\"\\nEvaluating Fine-Tuned Model...\")\n",
    "print(\"Llama3 Fine-Tuned Metrics:\", llama3_fine_tuned_metrics)\n",
    "print(\"DeepSeek Fine-Tuned Metrics:\", deepseek_fine_tuned_metrics)\n",
    "print(\"\\nComparing Fine-Tuned Models:\")\n",
    "print(\"Comparison Metrics:\", comparing_metrics)\n",
    "\n",
    "# Calculate average response times\n",
    "llama3_base_avg_time = sum(llama3_base_times) / len(llama3_base_times) if llama3_base_times else 0\n",
    "llama3_fine_tuned_avg_time = sum(llama3_fine_tuned_times) / len(llama3_fine_tuned_times) if llama3_fine_tuned_times else 0\n",
    "deepseek_base_avg_time = sum(deepseek_base_times) / len(deepseek_base_times) if deepseek_base_times else 0\n",
    "deepseek_fine_tuned_avg_time = sum(deepseek_fine_tuned_times) / len(deepseek_fine_tuned_times) if deepseek_fine_tuned_times else 0\n",
    "\n",
    "print(\"\\nAverage Response Times (seconds):\")\n",
    "print(f\"Llama3 Base: {llama3_base_avg_time:.4f}\")\n",
    "print(f\"Llama3 Fine-Tuned: {llama3_fine_tuned_avg_time:.4f}\")\n",
    "print(f\"DeepSeek Base: {deepseek_base_avg_time:.4f}\")\n",
    "print(f\"DeepSeek Fine-Tuned: {deepseek_fine_tuned_avg_time:.4f}\")\n"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}